In [0]:
%sql
CREATE TABLE IF NOT EXISTS kb_docs_chunks (
    id STRING NOT NULL,
    doc_title STRING,
    section STRING,
    text STRING,
    parent_doc_summary STRING,
    url STRING,
    updated_at DATE
)
USING DELTA
TBLPROPERTIES (
  delta.enableChangeDataFeed = true
);

INSERT INTO kb_docs_chunks VALUES
(
  'd01#c1',
  'Create Vector Search Index',
  'Delta Sync prerequisites',
  'Before you build an index from a Delta table, pick a primary key column, decide whether embeddings are computed by Databricks or precomputed, and ensure your Vector Search endpoint exists. Then create the index using Delta Sync.',
  'Step-by-step guide to creating a Vector Search index, including endpoint setup, Delta Sync configuration, primary key selection, and schema requirements.',
  '/kb/create-index',
  DATE '2025-12-01'
),
(
  'd02#c3',
  'Query a vector search index',
  'similarity_search',
  'Use index.similarity_search with query_text, query_type (ann or hybrid), columns, and num_results to retrieve the most relevant chunks from a vector search index.',
  'How to query vector search indexes using the Python SDK, REST API, and SQL, including filters and query types.',
  '/kb/query-index',
  DATE '2025-12-08'
),
(
  'd03#c2',
  'Query a vector search index',
  'rerank',
  'Reranking improves retrieval quality. Example query: query_text = "How to create a Vector Search index". Use columns_to_rerank to reorder retrieved documents based on semantic relevance.',
  'Explanation of vector search reranking, including columns_to_rerank, debug timing information, and failure handling.',
  '/kb/query-index#rerank',
  DATE '2025-12-08'
),
(
  'd04#c1',
  'Vector Search endpoint types',
  'Standard vs storage-optimized',
  'Standard endpoints use dictionary-style filters, while storage-optimized endpoints use SQL-like filter expressions when querying vector search indexes.',
  'Comparison of vector search endpoint types and their supported filtering syntax.',
  '/kb/endpoint-types',
  DATE '2025-10-12'
),
(
  'd05#c4',
  'Permissions for Vector Search',
  'Unity Catalog privileges',
  'To query a vector search index, users need USE CATALOG, USE SCHEMA, and SELECT privileges on the underlying Delta table and vector search index.',
  'Required Unity Catalog permissions and access control considerations for Vector Search.',
  '/kb/permissions',
  DATE '2025-09-20'
);


In [0]:
from databricks.vector_search.client import VectorSearchClient
from databricks.vector_search.reranker import DatabricksReranker

In [0]:
client = VectorSearchClient()
index = client.get_index(index_name="catalog.schema.docs_index")

results = index.similarity_search(
    query_text="How do I create a Vector Search index?",
    query_type="hybrid",
    num_results=10,  # final number of results returned
    columns=["id", "doc_title", "url", "text", "date"],

    # Reranker: columns used for scoring (not necessarily returned unless also in `columns`)
    reranker=DatabricksReranker(
        columns_to_rerank=["text", "parent_doc_summary", "doc_title"]
    ),
)


In [0]:
from databricks.vector_search.reranker import DatabricksReranker

results = index.similarity_search(
    query_text="How do I create a Vector Search index?",
    query_type="hybrid",
    columns=["id", "doc_title", "section", "url", "text"],
    num_results=5,
    reranker=DatabricksReranker(
        columns_to_rerank=["text", "parent_doc_summary", "doc_title"]
    ),
)
